# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [ ]:
# class LruCache(object):
#     def __init__(self, max_count):
#         <your code here>
# 
#     def __getitem__(self, key):
#         <your code here>
# 
#     def __setitem__(self, key, value):
#         <your code here>
# 
#     pass

#### Реализуйте декоратор

In [1]:
from functools import wraps
from collections import OrderedDict

def _advanced_hash(x):
    try:
        return hash(x)
    except TypeError:
        return hash((type(x), str(x)))


def cached(max_count=128):
    if type(max_count) != int:
        raise TypeError("'max_count' must be 'int'")
    if max_count <= 0:
        raise ValueError("'must_count' must be positive")

    def cache(func):
        results = OrderedDict()

        @wraps(func)
        def wrapper(*args, **kwargs):
            args_hash = tuple(_advanced_hash(x) for x in args)
            kwargs_hash = tuple((_advanced_hash(x), _advanced_hash(y)) for x, y in kwargs.items())
            final_hash = hash((hash(args_hash), hash(kwargs_hash)))

            if final_hash in results:
                results.move_to_end(final_hash)
                return results[final_hash]
            else:
                func_result = func(*args, **kwargs)
                results[final_hash] = func_result
                if len(results) > max_count:
                    results.popitem(last=False)
                return func_result

        return wrapper

    return cache


#### Проверьте использование декоратора

In [2]:
@cached(20)
def fact(n):
    print(f"'fact': call for {n}")
    if n < 2:
        return 1
    return fact(n - 1) * n

@cached(20)
def fact1(n):
    print(f"'fact1': call for {n}")
    if n < 2:
        return 1
    return fact1(n - 1) * n


In [3]:
fact(5)


'fact': call for 5
'fact': call for 4
'fact': call for 3
'fact': call for 2
'fact': call for 1


120

In [4]:
fact1(6)


'fact1': call for 6
'fact1': call for 5
'fact1': call for 4
'fact1': call for 3
'fact1': call for 2
'fact1': call for 1


720

In [5]:
fact(4)


24

In [6]:
fact1(10)


'fact1': call for 10
'fact1': call for 9
'fact1': call for 8
'fact1': call for 7


3628800

In [7]:
fact(25)


'fact': call for 25
'fact': call for 24
'fact': call for 23
'fact': call for 22
'fact': call for 21
'fact': call for 20
'fact': call for 19
'fact': call for 18
'fact': call for 17
'fact': call for 16
'fact': call for 15
'fact': call for 14
'fact': call for 13
'fact': call for 12
'fact': call for 11
'fact': call for 10
'fact': call for 9
'fact': call for 8
'fact': call for 7
'fact': call for 6


15511210043330985984000000

In [8]:
fact(25)


15511210043330985984000000

In [9]:
fact(10)


3628800

In [10]:
fact(5)


'fact': call for 5
'fact': call for 4
'fact': call for 3
'fact': call for 2
'fact': call for 1


120

In [11]:
@cached()
def to_str(x):
    sx = str(x)
    print(f"'to_str': call for {sx}")
    return sx


In [12]:
a = [1, 2, 3]
to_str(a)


'to_str': call for [1, 2, 3]


'[1, 2, 3]'

In [13]:
a.append(4)
to_str(a)


'to_str': call for [1, 2, 3, 4]


'[1, 2, 3, 4]'

In [14]:
del a[-1]
to_str(a)


'[1, 2, 3]'

#### Сравните свою реализацию с lru_cache из functools

In [15]:
from functools import lru_cache

@lru_cache(20)
def fact_lru(n):
    print(f"'fact_lru': call for {n}")
    if n < 2:
        return 1
    return fact_lru(n - 1) * n


In [16]:
fact_lru(25)

'fact_lru': call for 25
'fact_lru': call for 24
'fact_lru': call for 23
'fact_lru': call for 22
'fact_lru': call for 21
'fact_lru': call for 20
'fact_lru': call for 19
'fact_lru': call for 18
'fact_lru': call for 17
'fact_lru': call for 16
'fact_lru': call for 15
'fact_lru': call for 14
'fact_lru': call for 13
'fact_lru': call for 12
'fact_lru': call for 11
'fact_lru': call for 10
'fact_lru': call for 9
'fact_lru': call for 8
'fact_lru': call for 7
'fact_lru': call for 6
'fact_lru': call for 5
'fact_lru': call for 4
'fact_lru': call for 3
'fact_lru': call for 2
'fact_lru': call for 1


15511210043330985984000000

In [17]:
fact_lru(15)

1307674368000

In [18]:
fact_lru(5)

'fact_lru': call for 5
'fact_lru': call for 4
'fact_lru': call for 3
'fact_lru': call for 2
'fact_lru': call for 1


120

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

In [ ]:
<your code here>

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [19]:
from functools import wraps


def checked(*types: type):
    if not all(map(lambda o: isinstance(o, type), types)):
        raise TypeError("all arguments of 'checked' must be instances of 'type'")

    def checker(func):
        @wraps(func)
        def wrapper(*args):
            if len(args) != len(types):
                raise TypeError('wrong number of arguments')
            if not all(map(isinstance, args, types)):
                raise TypeError('wrong type')
            return func(*args)

        return wrapper

    return checker

#### Проверьте использование декоратора

In [20]:
from typing import List

# Пример
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    print("1st arg")
    print("    type:", type(a))
    print("    value:", a)
    print("2nd arg")
    print("    type:", type(b))
    print("    value:", b)
    print("3rd arg")
    print("    type:", type(c))
    print("    value:", c)


In [21]:
strange_func('hi', -5, [23, 'hello', 7j])


1st arg
    type: <class 'str'>
    value: hi
2nd arg
    type: <class 'int'>
    value: -5
3rd arg
    type: <class 'list'>
    value: [23, 'hello', 7j]


In [22]:
strange_func(3, 4, [5])


TypeError: wrong type

In [23]:
strange_func('3', 4)


TypeError: wrong number of arguments

In [24]:
strange_func('3', 4, [5], 7j)


TypeError: wrong number of arguments

In [25]:
@checked()
def empty_params_arg():
    print("empty")


In [26]:
empty_params_arg()


empty


In [27]:
empty_params_arg(5)


TypeError: wrong number of arguments

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [ ]:
<your code here>